1. Split into train and test data
2. Train model on train data normally
3. Take test data and duplicate into test prime 
4. Drop first visit from test prime data
5. Get predicted delta from test prime data. Compare to delta from test data. We know the difference (epsilon) because we dropped actual visits. What percent of time is test delta < test prime delta? 
6. Restrict it only to patients with lot of visits. Is this better?

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pickle

def clean_plot():
    ax = plt.subplot(111)    
    ax.spines["top"].set_visible(False)    
    ax.spines["bottom"].set_visible(False)    
    ax.spines["right"].set_visible(False)    
    ax.spines["left"].set_visible(False)    
    
    ax.get_xaxis().tick_bottom()    
    ax.get_yaxis().tick_left()   
    plt.grid()

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (10,6),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

In [3]:
import os, sys
project_root = os.path.abspath(os.path.dirname(os.getcwd()))
sys.path.append(project_root)

In [4]:
import sys
import torch

sys.path.append('.')
from data.load import chf
from data.data_utils import parse_data
from data.synthetic_data import load_piecewise_synthetic_data


sys.path.append('./model')
from models import Sublign
from run_experiments import get_hyperparameters


In [4]:
def make_test_prime(test_data_dict_raw, drop_first_T=1.):
    # drop first year
    test_data_dict = copy.deepcopy(test_data_dict_raw)
    eps_lst        = list()
    
    X = test_data_dict['obs_t_collect']
    Y = test_data_dict['Y_collect']
    M = test_data_dict['mask_collect']
    
    N_patients = X.shape[0]
    N_visits   = X.shape[1]
    
    for i in range(N_patients):
        eps_i = X[i,1,0] - X[i,0,0]
        
        first_visit = X[i,1,0]
        # move all visits down (essentially destroying the first visit)
        for j in range(N_visits-gap):
            
            X[i,j,0] = X[i,j+gap,0] - first_visit
            Y[i,j,:] = Y[i,j+gap,:]
            M[i,j,:] = M[i,j+gap,:]
        
        for g in range(1,gap+1):
            X[i,N_visits-g,0] = int(-1000)
            Y[i,N_visits-g,:] = int(-1000)
            M[i,N_visits-g,:] = 0.
        
        eps_lst.append(eps_i)
    return test_data_dict, eps_lst

In [9]:
import pandas as pd 

# Step 1: Load data
df = pd.read_csv('C:/Users/nss_1/clustering-interval-censored/model/data/result_4_long_format.csv', sep=';')

# Step 2: Filter to only V04, V06, V08
required_visits = ["V04", "V06", "V08"]
cog_cols = ["MCATOT", "NP1RTOT", "NP2PTOT", "NP3TOT", "SDMTOTAL"]
selected_columns = ["PATNO", "AGE_AT_VISIT", "FINAL_SEX_ENCODED", "COHORT", "subtype", "obs_time"] + cog_cols + ["EVENT_ID"]
import numpy as np

# Suppose cog_cols = ['A','B',…]
df[cog_cols] = df[cog_cols].replace(0, np.nan)
df = df.dropna(subset=cog_cols)
df = df[df["COHORT"].isin(["PD", "Healthy Control"])]
df['subtype'] = df['COHORT'].apply(lambda x: 0 if x == 'PD' else 1 if x == 'Healthy Control' else np.nan)
visit_encoding = {visit: i+1 for i, visit in enumerate(required_visits)}

# Assign obs_time using the mapping
df['obs_time'] = df['EVENT_ID'].map(visit_encoding)
df = df.sort_values(['PATNO', 'AGE_AT_VISIT']).reset_index(drop=True)
event_counts = df.groupby("PATNO")["EVENT_ID"].apply(set)

# Keep only IDs where all required visits are present
valid_ids = event_counts[event_counts.apply(lambda x: set(required_visits).issubset(x))].index

# Filter the DataFrame to keep only those IDs and the selected columns
df_filtered = df[df["PATNO"].isin(valid_ids) & df["EVENT_ID"].isin(required_visits)][selected_columns]


# Step 4: Drop any PATNO who has missing values in any of those columns
#def is_patient_valid(group):
    #return (len(group) == 3) and (not group[cog_cols].isnull().any().any())

#df = df.groupby("PATNO").filter(is_patient_valid)


# Step 7: Normalize
for col in cog_cols:
    max_val = df_filtered[col].max()
    if pd.notnull(max_val) and max_val != 0:
        df_filtered[col] = df_filtered[col] / max_val

# Final formatting
filtered_df = df_filtered[[
    "MCATOT", "NP1RTOT", "NP2PTOT", "NP3TOT", "SDMTOTAL",
    "subtype",
    "AGE_AT_VISIT",
    "PATNO",
    "obs_time"
]]
print(filtered_df["subtype"].value_counts())

0    588
1      9
Name: subtype, dtype: int64


c:\Users\nss_1\clustering-interval-censored\venv\lib\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (4,11,12,58,112,193,202,204) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
filtered_df

,MCATOT,NP1RTOT,NP2PTOT,NP3TOT,SDMTOTAL,subtype,AGE_AT_VISIT,PATNO,obs_time
6,0.933333,0.333333,0.034483,0.196721,0.573333,1,82.9,3008,1.0
7,0.900000,0.083333,0.068966,0.213115,0.466667,1,84.0,3008,2.0
8,0.800000,0.083333,0.068966,0.163934,0.440000,1,85.1,3008,3.0
9,0.866667,0.333333,0.517241,0.508197,0.706667,0,48.0,3010,1.0
10,0.933333,0.166667,0.689655,0.540984,0.813333,0,49.1,3010,2.0
...,...,...,...,...,...,...,...,...,...
1286,1.000000,0.083333,0.103448,0.278689,0.746667,0,67.0,114615,2.0
1287,0.933333,0.333333,0.137931,0.147541,0.626667,0,68.0,114615,3.0
1298,0.900000,0.083333,0.448276,0.737705,0.600000,0,68.2,116531,1.0
1299,1.000000,0.583333,0.551724,0.311475,0.626667,0,69.2,116531,2.0


In [ ]:
max_visits = 3
shuffle    = True
num_output_dims = data.shape[1] - 4
train_data_loader, train_data_dict, _, _, test_data_loader, test_data_dict, valid_pid, test_pid, unique_pid = parse_data(
            data.values, max_visits=max_visits, test_per=0.2, valid_per=0.2, shuffle=shuffle, device='cpu')
data_loader, collect_dict, unique_pid = parse_data(
            data.values, max_visits=max_visits, device="cpu")

b_vae, C, d_s, d_h, d_rnn, reg_type, lr = 0.01, 0.0, 10, 10, 20, 'l1', 0.1
epochs = 1000
device = 'cpu'
max_delta = 5.
learn_time = True

In [ ]:
model = Sublign(d_s, d_h, d_rnn, C=C, dim_biomarkers=num_output_dims,
                sigmoid=True, reg_type=reg_type, auto_delta=True,
                max_delta=max_delta, learn_time=learn_time)


In [ ]:
model.fit(
    train_data_loader,
    test_data_loader,
    epochs,
    lr=lr,
    verbose=True,
    fname='C:/Users/nss_1/clustering-interval-censored/model/runs/ppmi.pt',
    eval_freq=25
)


In [ ]:
results = model.score(train_data_dict, test_data_dict)
print('PPMI Test ARI: %.3f' % results['ari'])

# Extract results
subtypes = model.get_subtypes_datadict(collect_dict)
labels = model.get_labels(collect_dict)
deltas = model.get_deltas(collect_dict)


# Save
pickle.dump((labels, deltas, subtypes), open('C:/Users/nss_1/clustering-interval-censored/model/runs/ppmi_icml.pk', 'wb'))

In [11]:
import os
print(os.getcwd())

c:\Users\nss_1\clustering-interval-censored\model


In [6]:
import sys
import os

# Go up one directory from 'model/' to the project root
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)
sys.path.append(os.path.join(root_path, 'data'))
sys.path.append(os.path.join(root_path, 'model'))
sys.path.append(os.path.join(root_path, 'cross_validation'))

In [7]:
import numpy as np
import torch
import sys, os
from data.load import sigmoid, quadratic, load_data_format, parkinsons
from data.load import chf as load_chf
from data.data_utils import parse_data
from model.models import Sublign
import pickle

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from cross_validation.hpsearch import get_unsup_results

In [8]:
filtered_df

,MCATOT,NP1RTOT,NP2PTOT,NP3TOT,SDMTOTAL,subtype,AGE_AT_VISIT,PATNO,obs_time
0,1.000000,0.222222,0.103448,0.312500,0.367347,0,66.2,3001,1
1,1.000000,0.000000,0.068966,0.609375,0.428571,0,67.3,3001,2
2,0.966667,0.055556,0.206897,0.531250,0.489796,0,68.3,3001,3
3,0.933333,0.000000,0.172414,0.687500,0.346939,0,57.7,3003,1
4,0.900000,0.000000,0.241379,0.671875,0.397959,0,58.8,3003,2
...,...,...,...,...,...,...,...,...,...
2170,1.000000,0.055556,0.103448,0.265625,0.571429,0,67.0,114615,2
2171,0.933333,0.222222,0.137931,0.140625,0.479592,0,68.0,114615,3
2172,0.900000,0.055556,0.448276,0.703125,0.459184,0,68.2,116531,1
2173,1.000000,0.388889,0.551724,0.296875,0.479592,0,69.2,116531,2


In [ ]:
def run_cv(model='sublign', dataset='sigmoid', epochs=10, ppmi=False, search='nelbo'):
    import pickle
    import torch

    chf = False  # Fix missing variable
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Running on:', device)

    # Load dataset
    if ppmi:
        use_sigmoid = True
        data = filtered_df  # You manually prepared this earlier
        max_visits = 3
        train_loader, train_data_dict, valid_loader, valid_dict, p_ids, full_p_ids = parse_data(
            data.values, max_visits=max_visits, test_per=0.2
        )
        fname = 'runs/ppmi_hptune.pt'
    else:
        raise ValueError("Currently only ppmi=True is supported.")

    # Run hyperparameter search
    best_perf, best_ari, best_config, all_results = get_unsup_results(
        train_loader, train_data_dict, valid_loader, valid_dict,
        device, model, epochs=epochs, sigmoid=use_sigmoid,
        ppmi=ppmi, chf=chf, fname=fname, search=search
    )

    print(best_config, 'Best ARI: %.3f' % best_ari)

    # Save results
    fname = f"runs/{model}_ppmi.pkl"
    with open(fname, 'wb') as f:
        print('dumped pickle')
        pickle.dump(all_results, f)

In [ ]:
run_cv(model = 'sublign', dataset='sigmoid', epochs=1000, ppmi=True, search='nelbo') #, ppmi=True, search='nelbo')

Running on: cpu
Max visits: 3
<class 'model.models.Sublign'>
(False, 0.0, 1.0, 50, 10, 200, 'l1', 0.01)
z has nan in it
> c:\users\nss_1\clustering-interval-censored\model\models.py(746)get_subtypes()
    744             print('z has nan in it')
    745             import pdb; pdb.set_trace()
--> 746         km.fit(z)
    747         self.subtypes_km = km
    748 

